# SCRAPING SUPERMERCADOS:

## OBJETIVOS:
- Scrapear la página web de FACUA para conseguir un EDA sobre la variabilidad de los precios de tres productos principales: Aceite de Oliva, Aceite de Girasol y Leche.
- La comparación ha de ser entre los supermercados Alcampo, Carrefour, Dia, Eroski, Hipercor y Mercadona.
- Análisis de la Evolución de Precios
- Detección de Anomalías 
- Análisis de la Dispersión de Precios
- Comparación de Precios Promedio   
- Crear una base de datos SQL con la información.
- Visualización de datos


In [2]:
# Importamos las librerías que necesitamos
# !pip install selenium 
# !pip install bs4
# !pip install requests
# !pip install webdriver-manager
# !pip install pandas
# !pip install numpy
# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 


# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

#Librerias de soporte
#-------------------------------------------------------------------------

import sys
from src import soporte



Scrap supers 1h check
ir creando jsons por el camino check
Sacar tablas de cada uno
pasarlos a csv 30 min -- Viernes
crear bdd 3 h -- Viernes si puedo

queries en dbeaver
visualización

In [3]:
lista_supermercados = ["alcampo", "carrefour", "dia", "eroski", "hipercor", "mercadona"]

In [4]:
#Accedemos a la página web
driver = webdriver.Chrome()
url = "https://super.facua.org/"
driver.get(url)
driver.maximize_window()
sleep(2)

In [ ]:
#Accedemos al supermercado:
driver = webdriver.Chrome()
url = "https://super.facua.org/"
driver.get(url)
driver.maximize_window()
sleep(2)

try:
    driver.find_element("xpath",'//*[@id="rcc-confirm-button"]').click()
    print("Botón de aceptar cookies clicado con éxito.")
except:
    print("No se pudo encontrar el botón de aceptar cookies o hacer clic en él.")

# Automatically scroll the page

# Scroll down
driver.execute_script(f"window.scrollTo(0, {0,100});")
sleep(2)

#Click Mercadona
driver.find_element("xpath",'/html/body/section[1]/div/div[2]/div[1]/div/div[2]/div/a').click()
sleep(5)


#Scrapeamos aceite de girasol
# Scroll
driver.execute_script(f"window.scrollTo(0, {0,100});")
sleep(2)
#Click Aceite girasol
driver.find_element("xpath",'/html/body/section[1]/div/div[2]/div[1]/div/div[2]/div/a').click()
sleep(1)
#Aquí hay que scrapear todos los links de aceite de girasol !
get_url1 = driver.current_url
print("La url del aceite de girasol es:"+str(get_url1))

driver.back()
sleep(3)

#Click aceite de oliva
driver.find_element("xpath","/html/body/section[1]/div/div[2]/div[2]/div/div[2]/div/a").click()
sleep(1)
#Scrapeamos aceites de oliva
get_url2 = driver.current_url
print("La url del aceite de oliva es:"+str(get_url2))

driver.back()
sleep(3)

#Click leche
driver.find_element("xpath",'/html/body/section[1]/div/div[2]/div[3]/div/div[2]/div/a').click()
sleep(1)
#Scrapeamos leche
get_url3 = driver.current_url
print("La url de la leche es:"+str(get_url3))
driver.back()
sleep(3)

#Cerran2
driver.close()

In [ ]:
#Creamos un diccionario con los súpers y sus productos
diccionario_supers = {}
for supermercado in lista_supermercados:
    diccionario_gral = soporte.scrapeo_por_supermercado(supermercado)
    diccionario_supers[supermercado] = diccionario_gral

enlaces = [url for productos in diccionario_supers.values() for url in productos.values()]

In [44]:
#Sacamos los valores interiores
def aplicar_subproductos_a_productos(lista_enlaces_de_los_productos):
    dic_subs = {}
    for url in lista_enlaces_de_los_productos:
        url_producto = url
        res_producto = requests.get(url_producto)

        sopa_producto = BeautifulSoup(res_producto.content,"html.parser")

        lista_subproductos = []
        enlace_historico = sopa_producto.findAll("a", class_="btn-unirme btn-verde inline-block inline-block bg-primary border-primary font-semibold rounded-full")

# Extrae el valor del atributo href
        for valores in enlace_historico:
            lista_subproductos.append(valores.get("href"))
        
       
        dic_subs[url] = lista_subproductos
    return dic_subs

In [ ]:
sublista = aplicar_subproductos_a_productos(enlaces)

In [ ]:
sublista

In [ ]:
diccionario_supers

In [ ]:
#Sustituimos los productos en el diccionario por los subproductos

for sup in diccionario_supers:
    for cat in diccionario_supers[sup]: 
        print(sup,cat)
        diccionario_supers[sup][cat] = sublista[diccionario_supers[sup][cat]]

,alcampo,carrefour,dia,eroski,hipercor,mercadona
Aceite de girasol,[https://super.facua.org/alcampo/aceite-de-gir...,[https://super.facua.org/carrefour/aceite-de-g...,[https://super.facua.org/dia/aceite-de-girasol...,[https://super.facua.org/eroski/aceite-de-gira...,[https://super.facua.org/hipercor/aceite-de-gi...,[https://super.facua.org/mercadona/aceite-de-g...
Aceite de oliva,[https://super.facua.org/alcampo/aceite-de-oli...,[https://super.facua.org/carrefour/aceite-de-o...,[https://super.facua.org/dia/aceite-de-oliva/s...,[https://super.facua.org/eroski/aceite-de-oliv...,[https://super.facua.org/hipercor/aceite-de-ol...,[https://super.facua.org/mercadona/aceite-de-o...
Leche,[https://super.facua.org/alcampo/leche/enrique...,[https://super.facua.org/carrefour/leche/enriq...,[https://super.facua.org/dia/leche/enriquecida...,[https://super.facua.org/eroski/leche/enriquec...,[https://super.facua.org/hipercor/leche/enriqu...,[https://super.facua.org/mercadona/leche/enriq...


,alcampo,carrefour,dia,eroski,hipercor,mercadona
Aceite de girasol,https://super.facua.org/alcampo/aceite-de-gira...,[https://super.facua.org/carrefour/aceite-de-g...,[https://super.facua.org/dia/aceite-de-girasol...,[https://super.facua.org/eroski/aceite-de-gira...,[https://super.facua.org/hipercor/aceite-de-gi...,[https://super.facua.org/mercadona/aceite-de-g...
Aceite de girasol,https://super.facua.org/alcampo/aceite-de-gira...,[https://super.facua.org/carrefour/aceite-de-g...,[https://super.facua.org/dia/aceite-de-girasol...,[https://super.facua.org/eroski/aceite-de-gira...,[https://super.facua.org/hipercor/aceite-de-gi...,[https://super.facua.org/mercadona/aceite-de-g...
Aceite de girasol,https://super.facua.org/alcampo/aceite-de-gira...,[https://super.facua.org/carrefour/aceite-de-g...,[https://super.facua.org/dia/aceite-de-girasol...,[https://super.facua.org/eroski/aceite-de-gira...,[https://super.facua.org/hipercor/aceite-de-gi...,[https://super.facua.org/mercadona/aceite-de-g...
Aceite de girasol,https://super.facua.org/alcampo/aceite-de-gira...,[https://super.facua.org/carrefour/aceite-de-g...,[https://super.facua.org/dia/aceite-de-girasol...,[https://super.facua.org/eroski/aceite-de-gira...,[https://super.facua.org/hipercor/aceite-de-gi...,[https://super.facua.org/mercadona/aceite-de-g...
Aceite de girasol,https://super.facua.org/alcampo/aceite-de-gira...,[https://super.facua.org/carrefour/aceite-de-g...,[https://super.facua.org/dia/aceite-de-girasol...,[https://super.facua.org/eroski/aceite-de-gira...,[https://super.facua.org/hipercor/aceite-de-gi...,[https://super.facua.org/mercadona/aceite-de-g...
...,...,...,...,...,...,...
Leche,https://super.facua.org/alcampo/leche/tierra-d...,[https://super.facua.org/carrefour/leche/enriq...,[https://super.facua.org/dia/leche/enriquecida...,[https://super.facua.org/eroski/leche/enriquec...,[https://super.facua.org/hipercor/leche/enriqu...,[https://super.facua.org/mercadona/leche/enriq...
Leche,https://super.facua.org/alcampo/leche/tierra-d...,[https://super.facua.org/carrefour/leche/enriq...,[https://super.facua.org/dia/leche/enriquecida...,[https://super.facua.org/eroski/leche/enriquec...,[https://super.facua.org/hipercor/leche/enriqu...,[https://super.facua.org/mercadona/leche/enriq...
Leche,https://super.facua.org/alcampo/leche/tierra-d...,[https://super.facua.org/carrefour/leche/enriq...,[https://super.facua.org/dia/leche/enriquecida...,[https://super.facua.org/eroski/leche/enriquec...,[https://super.facua.org/hipercor/leche/enriqu...,[https://super.facua.org/mercadona/leche/enriq...
Leche,https://super.facua.org/alcampo/leche/tierra-d...,[https://super.facua.org/carrefour/leche/enriq...,[https://super.facua.org/dia/leche/enriquecida...,[https://super.facua.org/eroski/leche/enriquec...,[https://super.facua.org/hipercor/leche/enriqu...,[https://super.facua.org/mercadona/leche/enriq...
